In [9]:
!unzip plantdisease.zip -d plant_disease_data

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import torch
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from albumentations.pytorch import ToTensorV2
import albumentations as A

# For PyTorch transforms, wrap Albumentations
train_transforms = A.Compose([
    A.Resize(300, 300),
    A.RandomRotate90(),
    A.HorizontalFlip(),
    A.VerticalFlip(),
    A.RandomBrightnessContrast(),
    A.HueSaturationValue(),
    A.Normalize(),
    ToTensorV2()
])

valid_transforms = A.Compose([
    A.Resize(300, 300),
    A.Normalize(),
    ToTensorV2()
])

In [5]:
from torch.utils.data import Dataset
import cv2
import os

class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = cv2.imread(self.image_paths[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.labels[idx]

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        return image, label


In [18]:
from torchvision.datasets import ImageFolder
from torchvision import transforms

train_dataset = ImageFolder(
    root='plantdisease/PlantVillage',
    transform=transforms.Compose([
        transforms.Resize((300, 300)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ColorJitter(),
        transforms.ToTensor(),
    ])
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
NUM_CLASSES = len(train_dataset.classes)  # Automatically get the number of folders/classes
print(train_dataset.class_to_idx)
print(NUM_CLASSES)


{'Pepper__bell___Bacterial_spot': 0, 'Pepper__bell___healthy': 1, 'PlantVillage': 2, 'Potato___Early_blight': 3, 'Potato___Late_blight': 4, 'Potato___healthy': 5, 'Tomato_Bacterial_spot': 6, 'Tomato_Early_blight': 7, 'Tomato_Late_blight': 8, 'Tomato_Leaf_Mold': 9, 'Tomato_Septoria_leaf_spot': 10, 'Tomato_Spider_mites_Two_spotted_spider_mite': 11, 'Tomato__Target_Spot': 12, 'Tomato__Tomato_YellowLeaf__Curl_Virus': 13, 'Tomato__Tomato_mosaic_virus': 14, 'Tomato_healthy': 15}
16


In [14]:
import torchvision.models as models
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = len(train_dataset.classes)  # Automatically get the number of folders/classes


model = models.efficientnet_b3(pretrained=True)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, NUM_CLASSES)
model = model.to(device)
print("GPU:", device)


d:\UrbanNest\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\UrbanNest\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B3_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


GPU: cuda


In [ ]:
for param in model.parameters():
    param.requires_grad = True

optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Continue training for fine-tuning


In [ ]:
torch.save(model.state_dict(), 'leaf_model.pt')


In [15]:
import torch
import torchvision.models as models
import torch.nn as nn
# Load your model architecture first!
# For example, if you used MobileNetV2:
import torchvision.models as models

NUM_CLASSES = 15  # 👈 replace with your number of leaf classes

model = models.efficientnet_b3(pretrained=False)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)

# Load the weights
model.load_state_dict(torch.load("leaf_model.pt", map_location=torch.device("cpu")))
model.eval()


C:\Users\Asus\AppData\Local\Temp\ipykernel_17324\4250634206.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("leaf_model.pt", map_locati

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
            (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActiv

In [19]:
from PIL import Image
from torchvision import transforms

# Load image
image = Image.open("Tomato_early.jpg")  # 👈 replace with your test file path

# Apply same transforms as training!
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

input_tensor = transform(image).unsqueeze(0)  # add batch dimension


In [22]:
with torch.no_grad():
    outputs = model(input_tensor)
    _, predicted = torch.max(outputs, 1)
    probs = torch.nn.functional.softmax(outputs, dim=1)
    confidence = probs[0][predicted].item() * 100

print(f"Prediction: {predicted.item()} | Confidence: {confidence:.2f}%")



Prediction: 8 | Confidence: 64.10%
